# Decision Trees
- Simple Tree like structure, model makes a decision at every node
- Useful in simple tasks
- One of the most popular algorithm
- Easy explainability, easy to show how a decision process works!


### Why decision trees are popular?
- Easy to interpret and present
- Well defined Logic, mimic human level thought
- Random Forests, Ensembles of decision trees are more powerful classifiers
- Feature values are preferred to be categorical. If the values are continuous then they are     discretized prior to building the model.

### Build Decision Trees

Two common algorithms -
- CART (Classification and Regression Trees) → uses Gini Index(Classification) as metric.
- ID3 (Iterative Dichotomiser 3) → uses Entropy function and Information gain as metrics

In [1]:
import numpy as np
import pandas as pd 

In [3]:
data = pd.read_csv('titanic/train.csv')

In [5]:
data.head(n=10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [8]:
columns_to_drop=["PassengerId","Name","Ticket","Cabin","Embarked"]

In [10]:
data_clean=data.drop(columns=columns_to_drop)

In [13]:
data_clean.head(n=5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [17]:
from sklearn.preprocessing import LabelEncoder

le= LabelEncoder()
data_clean["Sex"]=le.fit_transform(data_clean["Sex"])

In [19]:
data_clean.head(n=5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


In [26]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
dtypes: float64(2), int64(5)
memory usage: 48.9 KB


In [27]:
data_clean= data_clean.fillna(data_clean["Age"].mean()) # NaN-> mean value of age

In [31]:
input_cols= ["Pclass","Sex","Age","SibSp","Parch","Fare"]
output_cols= ["Survived"]
x= data_clean[input_cols]
y= data_clean[output_cols]

print(x.shape,y.shape)
print(type(x))

(891, 6) (891, 1)
<class 'pandas.core.frame.DataFrame'>


In [32]:
# Define Entropy and Information Gain

In [35]:
def entropy(col):
    
    counts= np.unique(col,return_counts=True) # return tuples of unique elements
    N= float(col.shape[0])
    
    ent=0.0
    for ix in counts[1]:
        p= ix/N;
        ent+= (-1.0*p*np.log2(p))
        
    return ent
        


In [38]:
col= np.array([1,1,1,0,1,0,1])
entropy(col)

0.863120568566631

In [119]:
def divide_data(x_data, fkey, fval):
    # Work with Pandas DataFrames
    x_right = pd.DataFrame(columns=x_data.columns)
    x_left = pd.DataFrame(columns=x_data.columns)

    for ix in range(x_data.shape[0]):
        val = x_data.loc[ix, fkey]

        if val > fval:
            x_right.loc[ix] = x_data.loc[ix]
        else:
            x_left.loc[ix] = x_data.loc[ix]

    return x_left, x_right


In [152]:
x_left,x_right= divide_data(data_clean[:10],'Sex',0.5)
# print(data_clean[:10])
print(x_left  )
print(x_right)

   Survived  Pclass  Sex   Age  SibSp  Parch     Fare
1       1.0     1.0  0.0  38.0    1.0    0.0  71.2833
2       1.0     3.0  0.0  26.0    0.0    0.0   7.9250
3       1.0     1.0  0.0  35.0    1.0    0.0  53.1000
8       1.0     3.0  0.0  27.0    0.0    2.0  11.1333
9       1.0     2.0  0.0  14.0    1.0    0.0  30.0708
   Survived  Pclass  Sex        Age  SibSp  Parch     Fare
0       0.0     3.0  1.0  22.000000    1.0    0.0   7.2500
4       0.0     3.0  1.0  35.000000    0.0    0.0   8.0500
5       0.0     3.0  1.0  29.699118    0.0    0.0   8.4583
6       0.0     1.0  1.0  54.000000    0.0    0.0  51.8625
7       0.0     3.0  1.0   2.000000    3.0    1.0  21.0750


In [150]:
def information_gain(x_data,fkey,fval):
    
    left,right= divide_data(x_data,fkey,fval)
    
    # % of all samples are on left and right
    r= float(right.shape[0])/x_data.shape[0]
    l= float(left.shape[0])/x_data.shape[0]
                            
    # All examples come to one side!
    if left.shape[0]== 0 or right.shape[0]==0:
        return -1000000 # Min Information Gain
    
    i_gain= entropy(x_data.Survived)-(l*entropy(left.Survived)+r*entropy(right.Survived))
    return i_gain

In [151]:
# Testing
for fx in x.columns:
    print(fx)
    print(information_gain(data_clean,fx,data_clean[fx].mean()))

Pclass
0.07579362743608165
Sex
0.2176601066606142
Age
0.0008836151229467681
SibSp
0.009584541813400071
Parch
0.015380754493137694
Fare
0.042140692838995464


In [170]:
class DecisionTree:
    
    #constructor
    def __init__(self,depth=0,max_depth=5):
        self.left= None
        self.right= None
        self.fkey= None
        self.fval= None
        self.max_depth= max_depth
        self.depth= depth
        self.target= None
        
    def train(self,x_train):
        features= ["Pclass","Sex","Age","SibSp","Parch","Fare"]
        info_gains= []
        
        for ix in features:
            i_gain= information_gain(x_train,ix,x_train[ix].mean())
            info_gains.append(i_gain)
            
        self.fkey= features[np.argmax(info_gains)] # np.argmax return index of max element in the array
        self.fval= x_train[self.fkey].mean()
        print("Making Tree Features is ",self.fkey)
        
        #Split Data
        data_left,data_right= divide_data(x_train,self.fkey,self.fval)
        data_left= data_left.reset_index(drop=True)
        data_right= data_right.reset_index(drop=True)
        
        # Truly a left (or right) node
        if data_left.shape[0]==0 or data_right.shape[0]==0:
            if x_train.Survived.mean()>=0.5:
                self.target= "Survive"
            else:
                self.target= "Dead"
            return
        # Stop early when depth>=max depth
        if(self.depth>=self.max_depth):
            if x_train.Survived.mean()>=0.5:
                self.target= "Survive"
            else:
                self.target= "Dead"
            return
        
        # Recursive Case
        self.left= DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.left.train(data_left)
        
        self.right= DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.right.train(data_right)
        
        # keeping target at every node
        if x_train.Survived.mean()>=0.5:
                self.target= "Survive"
        else:
                self.target= "Dead"
        return
    
    def predict(self,test):
        if test[self.fkey]> self.fval:
            # go to right
            if self.right is None:
                return self.target
            return self.right.predict(test)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)
            
        
        

### Test Set Split

In [172]:
split= int(0.7*data_clean.shape[0])
train_data=data_clean[:split]
test_data=data_clean[split:]
test_data=test_data.reset_index(drop=True)

In [163]:
print(train_data.shape,test_data.shape)

(623, 7) (268, 7)


In [171]:
dt= DecisionTree()
dt.train(train_data)

Making Tree Features is  Sex
Making Tree Features is  Pclass
Making Tree Features is  Age
Making Tree Features is  SibSp
Making Tree Features is  Pclass
Making Tree Features is  Age
Making Tree Features is  Age
Making Tree Features is  SibSp
Making Tree Features is  Parch
Making Tree Features is  Pclass
Making Tree Features is  SibSp
Making Tree Features is  Fare
Making Tree Features is  Parch
Making Tree Features is  Age
Making Tree Features is  Pclass
Making Tree Features is  Age
Making Tree Features is  Age
Making Tree Features is  Parch
Making Tree Features is  SibSp
Making Tree Features is  Fare
Making Tree Features is  Age
Making Tree Features is  Age
Making Tree Features is  Fare
Making Tree Features is  Age
Making Tree Features is  Age
Making Tree Features is  Fare
Making Tree Features is  Age
Making Tree Features is  Parch
Making Tree Features is  Fare
Making Tree Features is  Fare
Making Tree Features is  Fare
Making Tree Features is  Age
Making Tree Features is  Fare
Making 

In [173]:
print(dt.fkey)
print(dt.fval)
print(dt.left.fkey)
print(dt.right.fkey)

Sex
0.6292134831460674
Pclass
Fare


In [175]:
y_pred= []
for ix in range(test_data.shape[0]):
    y_pred.append(dt.predict(test_data.loc[ix]))

In [177]:
y_pred

['Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'De

In [178]:
y_actual= test_data[output_cols]

In [180]:
print(y_actual)

     Survived
0           0
1           0
2           0
3           0
4           1
..        ...
263         0
264         1
265         0
266         1
267         0

[268 rows x 1 columns]


In [181]:
le= LabelEncoder()
y_pred= le.fit_transform(y_pred)


In [183]:
print(y_pred)

[0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 1 1 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0
 1 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 1 1 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 0 0 0
 0 1 0 1 0 0 0 0 1 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1
 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0
 0 0 0 0 1 0 0 1 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0 0 0 0 1 0 0 1 1 0 0 0 1 1 0
 0 0 0 0 0 1 0 0 0]


In [186]:
y_pred= np.array(y_pred).reshape((-1,1))
print(y_pred.shape)

(268, 1)


In [192]:
acc= np.sum(np.array(y_pred)==np.array(y_actual))/y_pred.shape[0]

In [193]:
print(acc)

0.8171641791044776
